In [176]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

os.getcwd()

/kaggle/input/submission/SampleSubmission.csv
/kaggle/input/dataedaz/data_EDA_full_features.csv


'/kaggle/working'

### Load the processed datset, test and tune models

In [178]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import seaborn as sns
import matplotlib.pyplot as plt
import time
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from itertools import combinations

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, KFold, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, auc, mean_squared_error, roc_curve, confusion_matrix, precision_score, recall_score, f1_score,\
log_loss

plt.rcParams['figure.figsize'] = (20,6)

In [180]:
import os

os.chdir('/kaggle/input')

In [182]:
#Load working dataset

data = pd.read_csv('../input/dataedaz/data_EDA_full_features.csv')

submission = pd.read_csv('submission/SampleSubmission.csv')
del(data['Unnamed: 0'])

#del(submission['Unnamed: 0'])

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [184]:
submission.head()

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0
2,F86J5PC X 8NN1,0
3,F86J5PC X 7POT,0
4,F86J5PC X 66FJ,0


In [186]:
prod_cols = list(data['PCODE'].unique())

In [188]:
os.chdir('/kaggle/working')

In [190]:
data.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,combiner,join_day,join_month,join_year,product_count,prod_proba,age_joined,age_user_current,age_current_group,age_joined_group,years_spent,join_week,branch_proba,year_proba,marital_proba,sex_proba,occu_proba,occucode_proba,age_proba,agejoin_proba,day_proba,week_proba,month_proba,year_spent_proba,max_prod,max_prod_proba,max_prod_2,max_prod_proba_2,max_prod_3,max_prod_proba_3,max_prod_4,max_prod_proba_4,max_prod_5,max_prod_proba_5,max_prod_6,max_prod_proba_6,P5DARIBP,P5DA8NN1,P5DA7POT,P5DA66FJ,P5DAGYSR,P5DASOP4,P5DARVSZ,P5DAPYUQ,P5DALJR9,P5DAN2MW,P5DAAHXO,P5DABSTQ,P5DAFM3X,P5DAK6QO,P5DAQBOL,P5DAJWFN,P5DAJZ9D,P5DAJ9JW,P5DAGHYX,P5DAECY3,RIBP8NN1,RIBP7POT,RIBP66FJ,RIBPGYSR,RIBPSOP4,RIBPRVSZ,RIBPPYUQ,RIBPLJR9,RIBPN2MW,RIBPAHXO,RIBPBSTQ,RIBPFM3X,RIBPK6QO,RIBPQBOL,RIBPJWFN,RIBPJZ9D,RIBPJ9JW,RIBPGHYX,RIBPECY3,8NN17POT,8NN166FJ,8NN1GYSR,8NN1SOP4,8NN1RVSZ,8NN1PYUQ,8NN1LJR9,8NN1N2MW,8NN1AHXO,8NN1BSTQ,8NN1FM3X,8NN1K6QO,8NN1QBOL,8NN1JWFN,8NN1JZ9D,8NN1J9JW,8NN1GHYX,8NN1ECY3,7POT66FJ,7POTGYSR,7POTSOP4,7POTRVSZ,7POTPYUQ,7POTLJR9,7POTN2MW,7POTAHXO,7POTBSTQ,7POTFM3X,7POTK6QO,7POTQBOL,7POTJWFN,7POTJZ9D,7POTJ9JW,7POTGHYX,7POTECY3,66FJGYSR,66FJSOP4,66FJRVSZ,66FJPYUQ,66FJLJR9,66FJN2MW,66FJAHXO,66FJBSTQ,66FJFM3X,66FJK6QO,66FJQBOL,66FJJWFN,66FJJZ9D,66FJJ9JW,66FJGHYX,66FJECY3,GYSRSOP4,GYSRRVSZ,GYSRPYUQ,GYSRLJR9,GYSRN2MW,GYSRAHXO,GYSRBSTQ,GYSRFM3X,GYSRK6QO,GYSRQBOL,GYSRJWFN,GYSRJZ9D,GYSRJ9JW,GYSRGHYX,GYSRECY3,SOP4RVSZ,SOP4PYUQ,SOP4LJR9,SOP4N2MW,SOP4AHXO,SOP4BSTQ,SOP4FM3X,SOP4K6QO,SOP4QBOL,SOP4JWFN,SOP4JZ9D,SOP4J9JW,SOP4GHYX,SOP4ECY3,RVSZPYUQ,RVSZLJR9,RVSZN2MW,RVSZAHXO,RVSZBSTQ,RVSZFM3X,RVSZK6QO,RVSZQBOL,RVSZJWFN,RVSZJZ9D,RVSZJ9JW,RVSZGHYX,RVSZECY3,PYUQLJR9,PYUQN2MW,PYUQAHXO,PYUQBSTQ,PYUQFM3X,PYUQK6QO,PYUQQBOL,PYUQJWFN,PYUQJZ9D,PYUQJ9JW,PYUQGHYX,PYUQECY3,LJR9N2MW,LJR9AHXO,LJR9BSTQ,LJR9FM3X,LJR9K6QO,LJR9QBOL,LJR9JWFN,LJR9JZ9D,LJR9J9JW,LJR9GHYX,LJR9ECY3,N2MWAHXO,N2MWBSTQ,N2MWFM3X,N2MWK6QO,N2MWQBOL,N2MWJWFN,N2MWJZ9D,N2MWJ9JW,N2MWGHYX,N2MWECY3,AHXOBSTQ,AHXOFM3X,AHXOK6QO,AHXOQBOL,AHXOJWFN,AHXOJZ9D,AHXOJ9JW,AHXOGHYX,AHXOECY3,BSTQFM3X,BSTQK6QO,BSTQQBOL,BSTQJWFN,BSTQJZ9D,BSTQJ9JW,BSTQGHYX,BSTQECY3,FM3XK6QO,FM3XQBOL,FM3XJWFN,FM3XJZ9D,FM3XJ9JW,FM3XGHYX,FM3XECY3,K6QOQBOL,K6QOJWFN,K6QOJZ9D,K6QOJ9JW,K6QOGHYX,K6QOECY3,QBOLJWFN,QBOLJZ9D,QBOLJ9JW,QBOLGHYX,QBOLECY3,JWFNJZ9D,JWFNJ9JW,JWFNGHYX,JWFNECY3,JZ9DJ9JW,JZ9DGHYX,JZ9DECY3,J9JWGHYX,J9JWECY3,GHYXECY3
0,ZJRPKUT,M,M,1971,X23B,0B60,JD7X,P5DA,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,x,5,jan,2018,1,0.780310,47.0,49,40s,40s,2,week1,0.003542,0.245023,0.781472,0.698476,0.019894,0.019517,0.331696,0.309406,0.126520,0.620605,0.999563,0.245023,7POT,0.282427,LJR9,0.263620,8NN1,0.203065,P5DA,0.149254,66FJ,0.097143,SOP4,0.074883,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4ZSFYBB,M,M,1983,E5SW,0B60,JD7X,P5DA,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,x,10,jan,2019,2,0.177434,36.0,37,30s,30s,1,week2,0.111615,0.305578,0.781472,0.698476,0.019894,0.019517,0.360150,0.374753,0.073425,0.379274,0.999563,0.305578,K6QO,0.891131,GHYX,0.613569,SOP4,0.574103,PYUQ,0.555942,ECY3,0.554976,66FJ,0.550476,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,2,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,

In [192]:
len(data)

76353

In [194]:
data = data.drop(['Label'], axis = 1)

data.dtypes

ID                           object
sex                          object
marital_status               object
birth_year                    int64
branch_code                  object
occupation_code              object
occupation_category_code     object
PCODE                        object
P5DA                          int64
RIBP                          int64
8NN1                          int64
7POT                          int64
66FJ                          int64
GYSR                          int64
SOP4                          int64
RVSZ                          int64
PYUQ                          int64
LJR9                          int64
N2MW                          int64
AHXO                          int64
BSTQ                          int64
FM3X                          int64
K6QO                          int64
QBOL                          int64
JWFN                          int64
JZ9D                          int64
J9JW                          int64
GHYX                        

In [196]:
data.columns

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE', 'P5DA', 'RIBP',
       ...
       'JWFNJZ9D', 'JWFNJ9JW', 'JWFNGHYX', 'JWFNECY3', 'JZ9DJ9JW', 'JZ9DGHYX',
       'JZ9DECY3', 'J9JWGHYX', 'J9JWECY3', 'GHYXECY3'],
      dtype='object', length=275)

In [198]:
#Extract categorical columns for label encoding

cat_cols = []

for col in data.columns:
    
    if data[col].dtype == 'O':
        
        cat_cols.append(col)

In [200]:
cat_cols = list(set(cat_cols)- set(['ID', 'PCODE', 'combiner']))

In [202]:
cat_cols

['join_month',
 'marital_status',
 'max_prod_4',
 'max_prod_6',
 'branch_code',
 'sex',
 'age_current_group',
 'max_prod_3',
 'age_joined_group',
 'max_prod',
 'max_prod_5',
 'occupation_category_code',
 'occupation_code',
 'max_prod_2',
 'join_week']

In [204]:
#Create Label Encoder for categorical features

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


for col in cat_cols:
    
    data[col] = le.fit_transform(data[col])

In [205]:
#Splitting dataset into train and test after label transformations

train = data[data['combiner'] == 'x']

test_set = data[data['combiner'] == 'y']

In [206]:
train.reset_index(drop = True, inplace = True)

test_set.reset_index(drop = True, inplace = True)

In [207]:
test_set.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,combiner,join_day,join_month,join_year,product_count,prod_proba,age_joined,age_user_current,age_current_group,age_joined_group,years_spent,join_week,branch_proba,year_proba,marital_proba,sex_proba,occu_proba,occucode_proba,age_proba,agejoin_proba,day_proba,week_proba,month_proba,year_spent_proba,max_prod,max_prod_proba,max_prod_2,max_prod_proba_2,max_prod_3,max_prod_proba_3,max_prod_4,max_prod_proba_4,max_prod_5,max_prod_proba_5,max_prod_6,max_prod_proba_6,P5DARIBP,P5DA8NN1,P5DA7POT,P5DA66FJ,P5DAGYSR,P5DASOP4,P5DARVSZ,P5DAPYUQ,P5DALJR9,P5DAN2MW,P5DAAHXO,P5DABSTQ,P5DAFM3X,P5DAK6QO,P5DAQBOL,P5DAJWFN,P5DAJZ9D,P5DAJ9JW,P5DAGHYX,P5DAECY3,RIBP8NN1,RIBP7POT,RIBP66FJ,RIBPGYSR,RIBPSOP4,RIBPRVSZ,RIBPPYUQ,RIBPLJR9,RIBPN2MW,RIBPAHXO,RIBPBSTQ,RIBPFM3X,RIBPK6QO,RIBPQBOL,RIBPJWFN,RIBPJZ9D,RIBPJ9JW,RIBPGHYX,RIBPECY3,8NN17POT,8NN166FJ,8NN1GYSR,8NN1SOP4,8NN1RVSZ,8NN1PYUQ,8NN1LJR9,8NN1N2MW,8NN1AHXO,8NN1BSTQ,8NN1FM3X,8NN1K6QO,8NN1QBOL,8NN1JWFN,8NN1JZ9D,8NN1J9JW,8NN1GHYX,8NN1ECY3,7POT66FJ,7POTGYSR,7POTSOP4,7POTRVSZ,7POTPYUQ,7POTLJR9,7POTN2MW,7POTAHXO,7POTBSTQ,7POTFM3X,7POTK6QO,7POTQBOL,7POTJWFN,7POTJZ9D,7POTJ9JW,7POTGHYX,7POTECY3,66FJGYSR,66FJSOP4,66FJRVSZ,66FJPYUQ,66FJLJR9,66FJN2MW,66FJAHXO,66FJBSTQ,66FJFM3X,66FJK6QO,66FJQBOL,66FJJWFN,66FJJZ9D,66FJJ9JW,66FJGHYX,66FJECY3,GYSRSOP4,GYSRRVSZ,GYSRPYUQ,GYSRLJR9,GYSRN2MW,GYSRAHXO,GYSRBSTQ,GYSRFM3X,GYSRK6QO,GYSRQBOL,GYSRJWFN,GYSRJZ9D,GYSRJ9JW,GYSRGHYX,GYSRECY3,SOP4RVSZ,SOP4PYUQ,SOP4LJR9,SOP4N2MW,SOP4AHXO,SOP4BSTQ,SOP4FM3X,SOP4K6QO,SOP4QBOL,SOP4JWFN,SOP4JZ9D,SOP4J9JW,SOP4GHYX,SOP4ECY3,RVSZPYUQ,RVSZLJR9,RVSZN2MW,RVSZAHXO,RVSZBSTQ,RVSZFM3X,RVSZK6QO,RVSZQBOL,RVSZJWFN,RVSZJZ9D,RVSZJ9JW,RVSZGHYX,RVSZECY3,PYUQLJR9,PYUQN2MW,PYUQAHXO,PYUQBSTQ,PYUQFM3X,PYUQK6QO,PYUQQBOL,PYUQJWFN,PYUQJZ9D,PYUQJ9JW,PYUQGHYX,PYUQECY3,LJR9N2MW,LJR9AHXO,LJR9BSTQ,LJR9FM3X,LJR9K6QO,LJR9QBOL,LJR9JWFN,LJR9JZ9D,LJR9J9JW,LJR9GHYX,LJR9ECY3,N2MWAHXO,N2MWBSTQ,N2MWFM3X,N2MWK6QO,N2MWQBOL,N2MWJWFN,N2MWJZ9D,N2MWJ9JW,N2MWGHYX,N2MWECY3,AHXOBSTQ,AHXOFM3X,AHXOK6QO,AHXOQBOL,AHXOJWFN,AHXOJZ9D,AHXOJ9JW,AHXOGHYX,AHXOECY3,BSTQFM3X,BSTQK6QO,BSTQQBOL,BSTQJWFN,BSTQJZ9D,BSTQJ9JW,BSTQGHYX,BSTQECY3,FM3XK6QO,FM3XQBOL,FM3XJWFN,FM3XJZ9D,FM3XJ9JW,FM3XGHYX,FM3XECY3,K6QOQBOL,K6QOJWFN,K6QOJZ9D,K6QOJ9JW,K6QOGHYX,K6QOECY3,QBOLJWFN,QBOLJZ9D,QBOLJ9JW,QBOLGHYX,QBOLECY3,JWFNJZ9D,JWFNJ9JW,JWFNGHYX,JWFNECY3,JZ9DJ9JW,JZ9DGHYX,JZ9DECY3,J9JWGHYX,J9JWECY3,GHYXECY3
0,F86J5PC,1,2,1984,4,96,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,y,12,0,2018,1,0.780310,34.0,36,1,1,2,1,0.124139,0.245023,0.781472,0.698476,0.194641,0.001145,0.360150,0.374753,0.086281,0.379274,0.999563,0.245023,12,0.769770,5,0.463033,20,0.436817,17,0.423338,0,0.407619,9,0.406536,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1,2,1996,0,128,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,y,10,0,2019,2,0.177434,23.0,24,0,0,1,1,0.040616,0.305578,0.781472,0.698476,0.194641,0.010519,0.113137,0.180338,0.073425,0.379274,0.999563,0.305578,10,0.801307,12,0.769770,5,0.745972,17,0.702183,18,0.663874,6,0.582891,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,2,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,RBAYUXZ,0,7,1968,11,19,5,0,0,0,0,0,

In [208]:
#products label encoder

le_prod = LabelEncoder()

train['PCODE'] = le_prod.fit_transform(train['PCODE'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [209]:
y_train = train[['PCODE']]

x_train = train.drop(['PCODE'], axis = 1)

In [210]:
x_train.columns

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       ...
       'JWFNJZ9D', 'JWFNJ9JW', 'JWFNGHYX', 'JWFNECY3', 'JZ9DJ9JW', 'JZ9DGHYX',
       'JZ9DECY3', 'J9JWGHYX', 'J9JWECY3', 'GHYXECY3'],
      dtype='object', length=274)

In [211]:
x_train.dtypes

ID                           object
sex                           int64
marital_status                int64
birth_year                    int64
branch_code                   int64
occupation_code               int64
occupation_category_code      int64
P5DA                          int64
RIBP                          int64
8NN1                          int64
7POT                          int64
66FJ                          int64
GYSR                          int64
SOP4                          int64
RVSZ                          int64
PYUQ                          int64
LJR9                          int64
N2MW                          int64
AHXO                          int64
BSTQ                          int64
FM3X                          int64
K6QO                          int64
QBOL                          int64
JWFN                          int64
JZ9D                          int64
J9JW                          int64
GHYX                          int64
ECY3                        

In [212]:
#drop the combiner tag, used it to know train and test

x_train = x_train.drop(['combiner'], axis = 1)

test_set = test_set.drop(['combiner'], axis = 1)

### MODEL BUILDING:

#### XGBOOST MODELLING

In [213]:
x_train.columns

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       ...
       'JWFNJZ9D', 'JWFNJ9JW', 'JWFNGHYX', 'JWFNECY3', 'JZ9DJ9JW', 'JZ9DGHYX',
       'JZ9DECY3', 'J9JWGHYX', 'J9JWECY3', 'GHYXECY3'],
      dtype='object', length=273)

### There are 273 features from feature engineering and original features. Feature selection is critical to prevent model being affected by noise features. We train model with duos and without, same with other features, monitoring cross-validation score to decide on optimum features.

In [214]:
x_train.columns[:60]

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3',
       'join_day', 'join_month', 'join_year', 'product_count', 'prod_proba',
       'age_joined', 'age_user_current', 'age_current_group',
       'age_joined_group', 'years_spent', 'join_week', 'branch_proba',
       'year_proba', 'marital_proba', 'sex_proba', 'occu_proba',
       'occucode_proba', 'age_proba', 'agejoin_proba', 'day_proba',
       'week_proba', 'month_proba', 'year_spent_proba', 'max_prod',
       'max_prod_proba', 'max_prod_2', 'max_prod_proba_2', 'max_prod_3',
       'max_prod_proba_3', 'max_prod_4', 'max_prod_proba_4', 'max_prod_5'],
      dtype='object')

In [215]:
x_train_feat = x_train.drop(columns= ['ID', 'GYSR', 'years_spent', 'product_count', 'year_spent_proba',\
                                     'join_month', 'month_proba', 'join_week', 'age_joined', \
                                     'age_user_current', 'birth_year', 'day_proba', 'week_proba', \
                                     'sex_proba', 'age_joined_group', 'agejoin_proba'\
                                     'max_prod_2', 'max_prod_proba_2','max_prod_3','max_prod_proba_3', \
                                      'max_prod_4', 'max_prod_proba_4', 'max_prod_5'], axis = 1)
                                     
,
          
                     
                           
                           
          

''

In [216]:
x_train_feat.columns[:60]

Index(['sex', 'marital_status', 'branch_code', 'occupation_code',
       'occupation_category_code', 'P5DA', 'RIBP', '8NN1', '7POT', '66FJ',
       'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO',
       'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'join_day', 'join_year',
       'age_current_group', 'branch_proba', 'year_proba', 'marital_proba',
       'occu_proba', 'occucode_proba', 'age_proba', 'max_prod',
       'max_prod_proba', 'max_prod_proba_5', 'max_prod_6', 'max_prod_proba_6',
       'P5DARIBP', 'P5DA8NN1', 'P5DA7POT', 'P5DA66FJ', 'P5DAGYSR', 'P5DASOP4',
       'P5DARVSZ', 'P5DAPYUQ', 'P5DALJR9', 'P5DAN2MW', 'P5DAAHXO', 'P5DABSTQ',
       'P5DAFM3X', 'P5DAK6QO', 'P5DAQBOL', 'P5DAJWFN', 'P5DAJZ9D', 'P5DAJ9JW',
       'P5DAGHYX', 'P5DAECY3', 'RIBP8NN1'],
      dtype='object')

In [217]:
prod_cols = list(data[data['combiner'] == 'x']['PCODE'].unique())

In [218]:
prod_cols

['P5DA',
 'RIBP',
 '8NN1',
 '7POT',
 '66FJ',
 'GYSR',
 'SOP4',
 'RVSZ',
 'PYUQ',
 'LJR9',
 'N2MW',
 'AHXO',
 'BSTQ',
 'FM3X',
 'K6QO',
 'QBOL',
 'JWFN',
 'JZ9D',
 'J9JW',
 'GHYX',
 'ECY3']

In [ ]:
#  params = {
#     'max_depth':[2, 4, 6, 8],
#     'learning_rate':[0.01, 0.02, 0.05],
#     'n_estimators':[700, 1000, 1200, 1500, 2000],
#     'reg_alpha': [0, 1, 5, 10], 
#      'gamma' : [0, 2, 5],
#     'min_child_weight' : [1, 2, 5, 10],
#  'subsample' : [0.2, 0.4, 0.8, 1.0], 
#  'colsample_bytree' : [0.2, 0.4, 0.6, 0.8, 1.0]}
                                                        
# rand = 40           
            
            
# from xgboost import XGBClassifier

# fold = StratifiedKFold(n_splits= 5, shuffle= True, random_state= rand)

# model = XGBClassifier(random_state = rand, tree_method = 'gpu_hist', gpu_id = 0)

# from sklearn.metrics import log_loss, make_scorer

# LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)


# cat_cv = RandomizedSearchCV(estimator = model, param_distributions = params, cv = fold, \
#                             scoring = 'balanced_accuracy')
    
# cat_cv.fit(x_train_feat , y_train['PCODE'].ravel())
    


# #print(cat_cv.grid_scores_)
     
# print('params ',cat_cv.best_params_)
     
     
# print('best score  ', cat_cv.best_score_)

In [ ]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
# print('params ',cat_cv.best_params_)

In [ ]:
# print('best score  ', cat_cv.best_score_)

#### Model Building: Single XGBOOST Model Cross-Validation

In [221]:
from xgboost import XGBClassifier

rand = 40

fold = StratifiedKFold(n_splits= 4, shuffle= True, random_state= rand)

i = 1

err = []

#Standardize
#scaler = StandardScaler()

for train_index, test_index in fold.split(x_train_feat, y_train):

    print('fold n°: ', i)  

    x_data, x_val = x_train_feat.iloc[train_index], x_train_feat.iloc[test_index]

    y_data, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
        
    #x_data, y_data = smote.fit_sample(x_data, y_data)
    
    # scaler.fit(x_data)
    
    # x_data = scaler.transform(x_data)
    # x_val = scaler.transform(x_val)
    
  
    xgb_model = XGBClassifier(n_estimators= 1200, learning_rate = 0.02, tree_method = "gpu_hist", \
                           gpu_id= 0, random_state = rand, objective = "multi : softprob", \
                          reg_alpha = 2)
    
    
#     'subsample': 0.4, 'reg_alpha': 5, 'n_estimators': 1200, 'min_child_weight': 1, \
#                     'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.8}
    
    
    
    xgb_model.fit(x_data, y_data['PCODE'].ravel(), eval_set= [(x_val, y_val['PCODE'].ravel())], \
              early_stopping_rounds= 800,\
              verbose= 100, eval_metric = "mlogloss")
        
    y_train_pred = xgb_model.predict_proba(x_data)

    y_test_pred = xgb_model.predict_proba(x_val)
        
    print('log_loss train', log_loss(y_data, y_train_pred))
    print('log_loss test', log_loss(y_val, y_test_pred))
    
    err.append(log_loss(y_val, y_test_pred))


    i = i + 1
        
print('Mean Log_loss test: ', np.mean(err))  


fold n°:  1
[0]	validation_0-mlogloss:2.88253
Will train until validation_0-mlogloss hasn't improved in 800 rounds.
[100]	validation_0-mlogloss:0.64750
[200]	validation_0-mlogloss:0.43970
[300]	validation_0-mlogloss:0.39529
[400]	validation_0-mlogloss:0.38179
[500]	validation_0-mlogloss:0.37616
[600]	validation_0-mlogloss:0.37323
[700]	validation_0-mlogloss:0.37157
[800]	validation_0-mlogloss:0.37051
[900]	validation_0-mlogloss:0.36963
[1000]	validation_0-mlogloss:0.36917
[1100]	validation_0-mlogloss:0.36884
[1199]	validation_0-mlogloss:0.36871
log_loss train 0.2509354278676462
log_loss test 0.36868244224732505
fold n°:  2
[0]	validation_0-mlogloss:2.88221
Will train until validation_0-mlogloss hasn't improved in 800 rounds.
[100]	validation_0-mlogloss:0.64798
[200]	validation_0-mlogloss:0.44079
[300]	validation_0-mlogloss:0.39597
[400]	validation_0-mlogloss:0.38202
[500]	validation_0-mlogloss:0.37632
[600]	validation_0-mlogloss:0.37367
[700]	validation_0-mlogloss:0.37171
[800]	validat

XGBOOST TRAINING ROUTINE: IN ORDER TO PREVENT RANDOMNESS WITH RESULTS USING XGB CLASSIFIER

In [222]:
y_train['PCODE'].nunique()

21

In [223]:
dtrain = xgb.DMatrix(x_train_feat, y_train)

param = { 'learning_rate':0.02, 'tree_method': 'gpu_hist', \
         'gpu_id' : 0, 'random_state': rand, 'objective' : 'multi:softprob', \
         'reg_alpha': 2.0, 'num_class' : 21}


model = xgb.train(param, dtrain, num_boost_round = 1200)

# model.save_model('/kaggle/working/xgb_0.026850.pkl')

In [224]:
#Examining feature importance scores
model.get_score(importance_type='gain')

{'marital_status': 16.036752967324862,
 'LJR9QBOL': 1.946392157424565,
 'max_prod_proba_5': 4.982981762058313,
 'SOP4K6QO': 4.193904755284406,
 'PYUQLJR9': 4.1135845079944895,
 'N2MWAHXO': 2.075292733419057,
 'occupation_code': 1.6800111649003426,
 'join_day': 1.8445392726487193,
 'join_year': 6.054382874576304,
 'branch_code': 3.123132760349036,
 'FM3XECY3': 1.635746844980403,
 '66FJSOP4': 4.468679774878926,
 '66FJK6QO': 1.8350760103852948,
 'RIBP66FJ': 5.559283772017094,
 '8NN1SOP4': 4.0856979471131725,
 'RIBPSOP4': 19.059294011159857,
 'occucode_proba': 1.9703243131442598,
 '8NN166FJ': 5.678051158777285,
 'max_prod_proba': 7.165172521183338,
 'RIBPPYUQ': 2.493378560423509,
 '7POTGHYX': 11.172236471610304,
 'max_prod_6': 2.906682054480478,
 'max_prod': 59.41730540248568,
 'age_proba': 1.310500559159668,
 'GYSRPYUQ': 3.110975037161553,
 '8NN1LJR9': 1.9762472234046922,
 'RIBPQBOL': 2.091735076593697,
 'N2MWQBOL': 2.114832300540887,
 'RIBPRVSZ': 5.021073713801774,
 '66FJPYUQ': 3.0072934

In [ ]:
#xgb.plot_importance(model, importance_type = 'gain')


#### CATBOOST MODELLING

In [ ]:
#  params = {
#     'max_depth':[2, 4, 6, 8],
#     'learning_rate':[0.01, 0.02, 0.05],
#     'n_estimators':[700, 1000, 1200, 1500, 2000],
#     'reg_alpha': [0, 1, 5, 10]}
                                                        
# rand = 40           
            
            
# from catboost import CatBoostClassifier

# model = CatBoostClassifier(random_state = rand, tree_method = 'gpu_hist', gpu_id = 0)


# cat_cv = RandomizedSearchCV(estimator = model, param_distributions = params, cv = 4)
    
# cat_cv.fit(x_train_feat , y_train['PCODE'].ravel())
    


# print(cat_cv.grid_scores_)
     
# print('params ',cat_cv.best_params_)
     
     
# print('best score  ', cat_cv.best_score_)

In [ ]:
# ##### !pip install catboost

# #Model trained on GPU

# from catboost import CatBoostClassifier

# model = CatBoostClassifier(iterations = 1200, learning_rate = 0.075, task_type = "GPU", \
#                            random_state = 10, silent = True)




# scores = -cross_val_score(model, x_train.drop(columns=['ID', 'age_user_current', 'years_spent', 'age_joined',\
#                                                        'birth_year']),\
#                           y_train, cv= 4, scoring = 'neg_log_loss')



# print("Scores:", scores)
# print("Mean:", scores.mean())
# print("Standard Deviation:", scores.std())

In [ ]:
# from catboost import CatBoostClassifier


# fold = StratifiedKFold(n_splits= 4, shuffle= True, random_state= 0)

# i = 1

# err = []

# #Standardize
# scaler = StandardScaler()

# for train_index, test_index in fold.split(x_train_feat, y_train):

#     print('fold n°: ', i)

#     x_data, x_val = x_train_feat.iloc[train_index], x_train_feat.iloc[test_index]

#     y_data, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
        
#     #x_data, y_data = smote.fit_sample(x_data, y_data)
    
#     # scaler.fit(x_data)
    
#     # x_data = scaler.transform(x_data)
#     # x_val = scaler.transform(x_val)
    
  
#     model = CatBoostClassifier(iterations= 1200, learning_rate = 0.075, task_type = "GPU", \
#                            devices='0:1', random_state = 40)

    
#     model.fit(x_data, y_data['PCODE'].ravel(), eval_set= [(x_val, y_val['PCODE'].ravel())], \
#               early_stopping_rounds= 300, verbose= 100)
        
#     y_train_pred = model.predict_proba(x_data)

#     y_test_pred = model.predict_proba(x_val)
        
#     print('log_loss train', log_loss(y_data, y_train_pred))
#     print('log_loss test', log_loss(y_val, y_test_pred))
    
#     err.append(log_loss(y_val, y_test_pred))


#     i = i + 1
        
# print('Mean Log_loss test: ', np.mean(err))  

    

LIGHTGBM Modelling

In [ ]:
# from lightgbm import LGBMClassifier

# model = LGBMClassifier(n_estimators= 700, learning_rate = 0.05, objective = 'multiclass', num_classes = 20)

# scores = -cross_val_score(model, x_train.drop(columns=['ID', 'join_day', 'birth_year', 'join_year', \
#                                                        'age_user_current']), y_train['PCODE'].ravel(), \
#                           cv= 4, scoring = 'neg_log_loss')


# print("Scores:", scores)
# print("Mean:", scores.mean())
# print("Standard Deviation:", scores.std())

In [ ]:
# from lightgbm import LGBMClassifier

# lig_cv = LGBMClassifier(n_estimators= 700, learning_rate = 0.1, objective = 'multiclassova', num_class = 21)

# scores = cross_val_score(lig_cv, x_train, y_train, cv= 3, scoring = "neg_log_loss")

# print("Scores:", scores)

# print("Mean:", scores.mean())

# print("Standard Deviation:", scores.std())

### Competition Test Set Prediction Routine:

In [225]:
import pickle
import os
os.getcwd()



'/kaggle/working'

In [226]:
test_set.columns

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE', 'P5DA', 'RIBP',
       ...
       'JWFNJZ9D', 'JWFNJ9JW', 'JWFNGHYX', 'JWFNECY3', 'JZ9DJ9JW', 'JZ9DGHYX',
       'JZ9DECY3', 'J9JWGHYX', 'J9JWECY3', 'GHYXECY3'],
      dtype='object', length=274)

In [227]:
#Ensure the test set is trained on same features as train

use_cols = list(x_train_feat.columns)

In [228]:
test_set.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,join_day,join_month,join_year,product_count,prod_proba,age_joined,age_user_current,age_current_group,age_joined_group,years_spent,join_week,branch_proba,year_proba,marital_proba,sex_proba,occu_proba,occucode_proba,age_proba,agejoin_proba,day_proba,week_proba,month_proba,year_spent_proba,max_prod,max_prod_proba,max_prod_2,max_prod_proba_2,max_prod_3,max_prod_proba_3,max_prod_4,max_prod_proba_4,max_prod_5,max_prod_proba_5,max_prod_6,max_prod_proba_6,P5DARIBP,P5DA8NN1,P5DA7POT,P5DA66FJ,P5DAGYSR,P5DASOP4,P5DARVSZ,P5DAPYUQ,P5DALJR9,P5DAN2MW,P5DAAHXO,P5DABSTQ,P5DAFM3X,P5DAK6QO,P5DAQBOL,P5DAJWFN,P5DAJZ9D,P5DAJ9JW,P5DAGHYX,P5DAECY3,RIBP8NN1,RIBP7POT,RIBP66FJ,RIBPGYSR,RIBPSOP4,RIBPRVSZ,RIBPPYUQ,RIBPLJR9,RIBPN2MW,RIBPAHXO,RIBPBSTQ,RIBPFM3X,RIBPK6QO,RIBPQBOL,RIBPJWFN,RIBPJZ9D,RIBPJ9JW,RIBPGHYX,RIBPECY3,8NN17POT,8NN166FJ,8NN1GYSR,8NN1SOP4,8NN1RVSZ,8NN1PYUQ,8NN1LJR9,8NN1N2MW,8NN1AHXO,8NN1BSTQ,8NN1FM3X,8NN1K6QO,8NN1QBOL,8NN1JWFN,8NN1JZ9D,8NN1J9JW,8NN1GHYX,8NN1ECY3,7POT66FJ,7POTGYSR,7POTSOP4,7POTRVSZ,7POTPYUQ,7POTLJR9,7POTN2MW,7POTAHXO,7POTBSTQ,7POTFM3X,7POTK6QO,7POTQBOL,7POTJWFN,7POTJZ9D,7POTJ9JW,7POTGHYX,7POTECY3,66FJGYSR,66FJSOP4,66FJRVSZ,66FJPYUQ,66FJLJR9,66FJN2MW,66FJAHXO,66FJBSTQ,66FJFM3X,66FJK6QO,66FJQBOL,66FJJWFN,66FJJZ9D,66FJJ9JW,66FJGHYX,66FJECY3,GYSRSOP4,GYSRRVSZ,GYSRPYUQ,GYSRLJR9,GYSRN2MW,GYSRAHXO,GYSRBSTQ,GYSRFM3X,GYSRK6QO,GYSRQBOL,GYSRJWFN,GYSRJZ9D,GYSRJ9JW,GYSRGHYX,GYSRECY3,SOP4RVSZ,SOP4PYUQ,SOP4LJR9,SOP4N2MW,SOP4AHXO,SOP4BSTQ,SOP4FM3X,SOP4K6QO,SOP4QBOL,SOP4JWFN,SOP4JZ9D,SOP4J9JW,SOP4GHYX,SOP4ECY3,RVSZPYUQ,RVSZLJR9,RVSZN2MW,RVSZAHXO,RVSZBSTQ,RVSZFM3X,RVSZK6QO,RVSZQBOL,RVSZJWFN,RVSZJZ9D,RVSZJ9JW,RVSZGHYX,RVSZECY3,PYUQLJR9,PYUQN2MW,PYUQAHXO,PYUQBSTQ,PYUQFM3X,PYUQK6QO,PYUQQBOL,PYUQJWFN,PYUQJZ9D,PYUQJ9JW,PYUQGHYX,PYUQECY3,LJR9N2MW,LJR9AHXO,LJR9BSTQ,LJR9FM3X,LJR9K6QO,LJR9QBOL,LJR9JWFN,LJR9JZ9D,LJR9J9JW,LJR9GHYX,LJR9ECY3,N2MWAHXO,N2MWBSTQ,N2MWFM3X,N2MWK6QO,N2MWQBOL,N2MWJWFN,N2MWJZ9D,N2MWJ9JW,N2MWGHYX,N2MWECY3,AHXOBSTQ,AHXOFM3X,AHXOK6QO,AHXOQBOL,AHXOJWFN,AHXOJZ9D,AHXOJ9JW,AHXOGHYX,AHXOECY3,BSTQFM3X,BSTQK6QO,BSTQQBOL,BSTQJWFN,BSTQJZ9D,BSTQJ9JW,BSTQGHYX,BSTQECY3,FM3XK6QO,FM3XQBOL,FM3XJWFN,FM3XJZ9D,FM3XJ9JW,FM3XGHYX,FM3XECY3,K6QOQBOL,K6QOJWFN,K6QOJZ9D,K6QOJ9JW,K6QOGHYX,K6QOECY3,QBOLJWFN,QBOLJZ9D,QBOLJ9JW,QBOLGHYX,QBOLECY3,JWFNJZ9D,JWFNJ9JW,JWFNGHYX,JWFNECY3,JZ9DJ9JW,JZ9DGHYX,JZ9DECY3,J9JWGHYX,J9JWECY3,GHYXECY3
0,F86J5PC,1,2,1984,4,96,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,2018,1,0.780310,34.0,36,1,1,2,1,0.124139,0.245023,0.781472,0.698476,0.194641,0.001145,0.360150,0.374753,0.086281,0.379274,0.999563,0.245023,12,0.769770,5,0.463033,20,0.436817,17,0.423338,0,0.407619,9,0.406536,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1,2,1996,0,128,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,10,0,2019,2,0.177434,23.0,24,0,0,1,1,0.040616,0.305578,0.781472,0.698476,0.194641,0.010519,0.113137,0.180338,0.073425,0.379274,0.999563,0.305578,10,0.801307,12,0.769770,5,0.745972,17,0.702183,18,0.663874,6,0.582891,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,2,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,RBAYUXZ,0,7,1968,11,19,5,0,0,0,0,0,0,0,0,1,0,0,0

In [229]:
#Predict the probabilities of the different products

test_xgb = test_set[use_cols]

dtest = xgb.DMatrix(test_xgb)


proba = model.predict(dtest)



In [230]:
proba

array([[9.3068069e-05, 5.7474994e-05, 1.4304461e-05, ..., 2.5089039e-04,
        2.1506125e-05, 1.3119997e-04],
       [1.5424232e-03, 2.8543593e-04, 1.2444581e-04, ..., 1.6023160e-03,
        1.4978276e-04, 3.5522701e-04],
       [6.7175512e-04, 1.8888782e-04, 6.2114222e-04, ..., 1.5281284e-03,
        2.1674265e-05, 1.5216542e-03],
       ...,
       [6.1829832e-07, 7.9058492e-07, 1.1993373e-07, ..., 2.1452813e-06,
        9.9996424e-01, 6.5917902e-07],
       [4.0409374e-03, 8.2956220e-04, 7.3417630e-03, ..., 1.1136443e-02,
        4.0320747e-02, 5.7290029e-03],
       [3.4268603e-05, 1.4394800e-06, 2.0711695e-06, ..., 2.5620975e-05,
        9.9987268e-01, 3.1948890e-05]], dtype=float32)

In [231]:
y_test = pd.DataFrame(proba)
y_test.columns = le_prod.inverse_transform(y_test.columns)

In [232]:
len(y_test.columns)

21

In [233]:
#test set contains the prediction rows

test = test_set.copy()

In [234]:
len(test)

10000

In [235]:
#set the original product rows in test set to the predicted product rows

for c in y_test.columns:
    
    test[c] = y_test[c]

In [236]:
test.isnull().sum()

ID                          0
sex                         0
marital_status              0
birth_year                  0
branch_code                 0
occupation_code             0
occupation_category_code    0
PCODE                       0
P5DA                        0
RIBP                        0
8NN1                        0
7POT                        0
66FJ                        0
GYSR                        0
SOP4                        0
RVSZ                        0
PYUQ                        0
LJR9                        0
N2MW                        0
AHXO                        0
BSTQ                        0
FM3X                        0
K6QO                        0
QBOL                        0
JWFN                        0
JZ9D                        0
J9JW                        0
GHYX                        0
ECY3                        0
join_day                    0
join_month                  0
join_year                   0
product_count               0
prod_proba

In [237]:
test.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,join_day,join_month,join_year,product_count,prod_proba,age_joined,age_user_current,age_current_group,age_joined_group,years_spent,join_week,branch_proba,year_proba,marital_proba,sex_proba,occu_proba,occucode_proba,age_proba,agejoin_proba,day_proba,week_proba,month_proba,year_spent_proba,max_prod,max_prod_proba,max_prod_2,max_prod_proba_2,max_prod_3,max_prod_proba_3,max_prod_4,max_prod_proba_4,max_prod_5,max_prod_proba_5,max_prod_6,max_prod_proba_6,P5DARIBP,P5DA8NN1,P5DA7POT,P5DA66FJ,P5DAGYSR,P5DASOP4,P5DARVSZ,P5DAPYUQ,P5DALJR9,P5DAN2MW,P5DAAHXO,P5DABSTQ,P5DAFM3X,P5DAK6QO,P5DAQBOL,P5DAJWFN,P5DAJZ9D,P5DAJ9JW,P5DAGHYX,P5DAECY3,RIBP8NN1,RIBP7POT,RIBP66FJ,RIBPGYSR,RIBPSOP4,RIBPRVSZ,RIBPPYUQ,RIBPLJR9,RIBPN2MW,RIBPAHXO,RIBPBSTQ,RIBPFM3X,RIBPK6QO,RIBPQBOL,RIBPJWFN,RIBPJZ9D,RIBPJ9JW,RIBPGHYX,RIBPECY3,8NN17POT,8NN166FJ,8NN1GYSR,8NN1SOP4,8NN1RVSZ,8NN1PYUQ,8NN1LJR9,8NN1N2MW,8NN1AHXO,8NN1BSTQ,8NN1FM3X,8NN1K6QO,8NN1QBOL,8NN1JWFN,8NN1JZ9D,8NN1J9JW,8NN1GHYX,8NN1ECY3,7POT66FJ,7POTGYSR,7POTSOP4,7POTRVSZ,7POTPYUQ,7POTLJR9,7POTN2MW,7POTAHXO,7POTBSTQ,7POTFM3X,7POTK6QO,7POTQBOL,7POTJWFN,7POTJZ9D,7POTJ9JW,7POTGHYX,7POTECY3,66FJGYSR,66FJSOP4,66FJRVSZ,66FJPYUQ,66FJLJR9,66FJN2MW,66FJAHXO,66FJBSTQ,66FJFM3X,66FJK6QO,66FJQBOL,66FJJWFN,66FJJZ9D,66FJJ9JW,66FJGHYX,66FJECY3,GYSRSOP4,GYSRRVSZ,GYSRPYUQ,GYSRLJR9,GYSRN2MW,GYSRAHXO,GYSRBSTQ,GYSRFM3X,GYSRK6QO,GYSRQBOL,GYSRJWFN,GYSRJZ9D,GYSRJ9JW,GYSRGHYX,GYSRECY3,SOP4RVSZ,SOP4PYUQ,SOP4LJR9,SOP4N2MW,SOP4AHXO,SOP4BSTQ,SOP4FM3X,SOP4K6QO,SOP4QBOL,SOP4JWFN,SOP4JZ9D,SOP4J9JW,SOP4GHYX,SOP4ECY3,RVSZPYUQ,RVSZLJR9,RVSZN2MW,RVSZAHXO,RVSZBSTQ,RVSZFM3X,RVSZK6QO,RVSZQBOL,RVSZJWFN,RVSZJZ9D,RVSZJ9JW,RVSZGHYX,RVSZECY3,PYUQLJR9,PYUQN2MW,PYUQAHXO,PYUQBSTQ,PYUQFM3X,PYUQK6QO,PYUQQBOL,PYUQJWFN,PYUQJZ9D,PYUQJ9JW,PYUQGHYX,PYUQECY3,LJR9N2MW,LJR9AHXO,LJR9BSTQ,LJR9FM3X,LJR9K6QO,LJR9QBOL,LJR9JWFN,LJR9JZ9D,LJR9J9JW,LJR9GHYX,LJR9ECY3,N2MWAHXO,N2MWBSTQ,N2MWFM3X,N2MWK6QO,N2MWQBOL,N2MWJWFN,N2MWJZ9D,N2MWJ9JW,N2MWGHYX,N2MWECY3,AHXOBSTQ,AHXOFM3X,AHXOK6QO,AHXOQBOL,AHXOJWFN,AHXOJZ9D,AHXOJ9JW,AHXOGHYX,AHXOECY3,BSTQFM3X,BSTQK6QO,BSTQQBOL,BSTQJWFN,BSTQJZ9D,BSTQJ9JW,BSTQGHYX,BSTQECY3,FM3XK6QO,FM3XQBOL,FM3XJWFN,FM3XJZ9D,FM3XJ9JW,FM3XGHYX,FM3XECY3,K6QOQBOL,K6QOJWFN,K6QOJZ9D,K6QOJ9JW,K6QOGHYX,K6QOECY3,QBOLJWFN,QBOLJZ9D,QBOLJ9JW,QBOLGHYX,QBOLECY3,JWFNJZ9D,JWFNJ9JW,JWFNGHYX,JWFNECY3,JZ9DJ9JW,JZ9DGHYX,JZ9DECY3,J9JWGHYX,J9JWECY3,GHYXECY3
0,F86J5PC,1,2,1984,4,96,1,0,0.000014,0.000251,1.430446e-05,0.000057,0.000093,4.138028e-05,0.000131,0.000022,0.001418,0.000058,0.000277,0.000147,0.000013,7.162828e-06,0.976648,0.011558,0.000163,0.000019,0.000025,0.006204,0.002839,12,0,2018,1,0.780310,34.0,36,1,1,2,1,0.124139,0.245023,0.781472,0.698476,0.194641,0.001145,0.360150,0.374753,0.086281,0.379274,0.999563,0.245023,12,0.769770,5,0.463033,20,0.436817,17,0.423338,0,0.407619,9,0.406536,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1,2,1996,0,128,1,0,0.000208,0.001602,1.244458e-04,0.000285,0.001542,8.687120e-05,0.000355,0.000150,0.010656,0.000432,0.000278,0.000531,0.000062,6.011009e-05,0.000369,0.427004,0.041862,0.000124,0.000099,0.285148,0.229020,10,0,2019,2,0.177434,23.0,24,0,0,1,1,0.040616,0.305578,0.781472,0.698476,0.194641,0.010519,0.113137,0.180338,0.073425,0.379274,0.999563,0.305578,10,0.801307,12,0.769770,5,0.745972,17,0.702183,18,0.663874,6,0.582891,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [238]:
test.columns

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE', 'P5DA', 'RIBP',
       ...
       'JWFNJZ9D', 'JWFNJ9JW', 'JWFNGHYX', 'JWFNECY3', 'JZ9DJ9JW', 'JZ9DGHYX',
       'JZ9DECY3', 'J9JWGHYX', 'J9JWECY3', 'GHYXECY3'],
      dtype='object', length=274)

In [239]:
test.columns[0]

'ID'

In [240]:
#selecting just the PCODE and Label probabilities

test = test.melt(id_vars=test.columns[:1], value_vars= test.columns[8:29], var_name = "PCODE", value_name="Label" )

In [241]:
test.head()

,ID,PCODE,Label
0,F86J5PC,P5DA,0.000014
1,H6141K3,P5DA,0.000208
2,RBAYUXZ,P5DA,0.000048
3,KCBILBQ,P5DA,0.000005
4,LSEC1ZJ,P5DA,0.000002


In [242]:
#submission file is expected to have 1 for certain ids, we extract the ids

true_ids = list(submission[submission['Label'] == 1]['ID X PCODE'])

In [243]:
test.head()

,ID,PCODE,Label
0,F86J5PC,P5DA,0.000014
1,H6141K3,P5DA,0.000208
2,RBAYUXZ,P5DA,0.000048
3,KCBILBQ,P5DA,0.000005
4,LSEC1ZJ,P5DA,0.000002


In [244]:
def product_to_label(df):
    
    """
        Function to loop through each user and PCODE, extract the prediction at that PCODE and label
        Set it as a list
    """
    
    PCODE = df['PCODE']
    
    label = df['Label']
        
        
    return [df["ID"] + " X " + PCODE, label]
    
    

    


In [245]:
#the list of ID X PCODE & Label combinations

lis = test.apply(product_to_label,  axis = 1)

In [246]:
test.head()

,ID,PCODE,Label
0,F86J5PC,P5DA,0.000014
1,H6141K3,P5DA,0.000208
2,RBAYUXZ,P5DA,0.000048
3,KCBILBQ,P5DA,0.000005
4,LSEC1ZJ,P5DA,0.000002


In [247]:
#append to a list for DataFrame format

prod_list = []

for l in lis:
    
    prod_list.append(l)

In [248]:
#Create DataFrame of the required format

test = pd.DataFrame(prod_list)

test.columns = ['ID X PCODE', 'Label']

In [249]:
test.head(100)

,ID X PCODE,Label
0,F86J5PC X P5DA,1.448976e-05
1,H6141K3 X P5DA,2.083644e-04
2,RBAYUXZ X P5DA,4.790356e-05
3,KCBILBQ X P5DA,5.438483e-06
4,LSEC1ZJ X P5DA,2.113119e-06
5,EK65KRH X P5DA,4.113310e-05
6,M3N7TLU X P5DA,8.253906e-05
7,QAC6HOX X P5DA,2.559036e-05
8,T8B9JK0 X P5DA,9.668017e-05
9,62M79HM X P5DA,1.358481e-07


In [250]:
def convert_one(df, ids):
    
    if df['ID X PCODE'] in ids:
        
        return 1.0
    
    else:
        
        return df['Label']
        
        

In [251]:
#Apply function to use 1.0 for columns already in submission file

test['Label'] = test.apply(convert_one, args = [true_ids], axis = 1)

In [252]:
submission.head()

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0
2,F86J5PC X 8NN1,0
3,F86J5PC X 7POT,0
4,F86J5PC X 66FJ,0


In [253]:
os.chdir('/kaggle/working')

In [254]:
#Create Submission

test.to_csv('/kaggle/working/damola_submission_final_2.csv', index = False)